In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from sklearn import metrics
import matplotlib.pyplot as plt
from functools import reduce
import math
from collections import Counter
import csv
from datetime import datetime
import re
import copy
import json
from datetime import timedelta
from operator import itemgetter
%matplotlib inline

In [2]:
def now():
    return str(datetime.now().time())[:8]
def pr(strToPrint):
    print(now() + ' '+ strToPrint)

from IPython.display import Audio
sound_file = 'beep.wav'
# pd.set_option('display.max_colwidth', -1)

## Importing data

Importing a sample of the dataset:

In [3]:
pickle_filename = os.path.join('data','head_100k_pickle.pkl')
tw = pd.read_pickle(pickle_filename)

Importing the whole dataset

In [4]:
columns_header = ['id', 'userId', 'createdAt', 'text', 'longitude', 'latitude', 'placeId',
                  'inReplyTo', 'source', 'truncated', 'placeLatitude', 'placeLongitude', 'sourceName', 'sourceUrl',
                 'userName', 'screenName', 'followersCount', 'friendsCount', 'statusesCount',
                 'userLocation']

filename = os.path.join('data','twex.tsv') # 'sample.tsv')
pr('Starting to read file... (3 min)')
tw = pd.read_csv(filename, sep='\t', encoding='utf-8', escapechar='\\', names=columns_header,
                      quoting=csv.QUOTE_NONE, na_values='N', header=None)

pr('File is loaded!')
# Audio(url=sound_file, autoplay=True)

10:01:26 Starting to read file... (3 min)


C:\Users\Daniel\Anaconda3\envs\ADA-kernel\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,5,6,8,9,10,11,12,13,14,15,16,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


10:04:03 File is loaded!


In [5]:
tw.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,1.7341e+07,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508.0,9016.0,"Bern, Switzerland"
1,9514846412,7.19828e+06,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703.0,4687.0,"Zurich, Switzerland"


## Extrating hashtags

In [6]:
def extract_hashtags(text):
    ht_list = re.findall(r"#(\w+)", text)
    non_empty_hts = list(filter((lambda ht: ht != []), ht_list))
    lowerCharList = [ht.lower() for ht in non_empty_hts]
    return lowerCharList

In [7]:
pr('Making hashtags... (2 min)')
tw['hashtag'] = np.nan
tw.hashtag = tw.text.apply(lambda x: extract_hashtags(str(x)))
twh = tw.ix[tw.hashtag.apply(lambda x: len(x) != 0)]
pr('Done.')

10:04:03 Making hashtags... (2 min)
10:05:47 Done.


In [8]:
twh.head(2)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,...,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
8,9519737890,1.46579e+07,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,...,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
10,9521789689,9.96202e+06,2010-02-23 11:28:27,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,...,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


## Cleaning data and making date index

In [9]:
tw1 = twh.dropna(axis=0, how='any', subset=['text', 'createdAt'])
print('The data have been reduced from {} tweets to {} tweets.'.format(len(twh), len(tw1)))

The data have been reduced from 3875280 tweets to 3875280 tweets.


In [10]:
pr('Removing bad dates...')
twhCleanDate = tw1[tw1['createdAt'].str.len() == 19]
pr('Finished.')

10:05:50 Removing bad dates...
10:05:53 Finished.


In [11]:
pr('Starting to examine dates...')
import warnings
warnings.filterwarnings('ignore')
datetime_serie = twhCleanDate['createdAt'].convert_objects(convert_dates='coerce')
dateNotConvertible = datetime_serie[pd.isnull(datetime_serie)]
warnings.filterwarnings('default')
pr('There are {} dates that cannot be transformed.'.format(len(dateNotConvertible)))

10:05:53 Starting to examine dates...
10:05:55 There are 0 dates that cannot be transformed.


In [12]:
pr('Starting copy...') # (to avoid transformation problems)
tw5 = twhCleanDate.copy()
pr('Converting to datetime...')
tw5['createdAt'] = pd.to_datetime(twhCleanDate['createdAt'])
pr('Setting up new indices...')
tw5.index = tw5['createdAt']
pr('Deleting old "createdAt" column...')
del tw5['createdAt']
pr('Done!')
tw5.head(2)

10:05:55 Starting copy...
10:05:56 Converting to datetime...
10:05:59 Setting up new indices...
10:05:59 Deleting old "createdAt" column...
10:05:59 Done!


,id,userId,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation,hashtag
createdAt,,,,,,,,,,,,,,,,,,,,
2010-02-23 09:59:41,9519737890,1.46579e+07,"Magic spells run off after midnight, I guess s...",6.13870,46.175,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277.0,2885.0,"Geneva, Switzerland",[fb]
2010-02-23 11:28:27,9521789689,9.96202e+06,"Limitas of public transportation! No taxi, rai...",6.33641,46.4631,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Thomas Winter,thwinter,1070,1359.0,3349.0,Hettlingen CH / SanJose Ca,[yam]


In [13]:
tw5['hashtag'][:6]

createdAt
2010-02-23 09:59:41               [fb]
2010-02-23 11:28:27              [yam]
2010-02-23 17:47:11          [24, vfb]
2010-02-23 18:19:03    [iphoneography]
2010-02-23 18:31:46     [partnermonth]
2010-02-24 06:09:23      [insider, fb]
Name: hashtag, dtype: object

## Let's put one hashtag per row

We will make a dataframe with one row = one hashtag. This will be done by going through the dataframe, and making in parallel a list of rows (with 1 hashtag per row) that needs to be added to the old dataframe.

In [14]:
addedHashtagsRowsList = []
def multiplyHashtagRows(row, columns):
    '''
    Examine each row. If there are multiple hashtags, it will return the first one.
    (so the first one will replace the list of hashtags in the df). Then for all the next ones,
    it will make a copy of the row in the addedHashtagsRowsList, (in a dictionary format).
    So this dictionary can in the end be transformed in a DF and added to the original DF.
    (The speed is increased a lot by doing it this way!)
    '''
    htList = row.hashtag
    if len(htList) > 1:
        ## Making the dictionary
        addedHashtag = {}
        addedHashtag['createdAt'] = row.name #the df index
        for col in columns:
            addedHashtag[col] = row[col]
        ## Copying the dict for each hashtag
        i = 1
        while i < len(htList) :
            deepCopy = copy.deepcopy(addedHashtag)
            deepCopy['hashtag'] = htList[i]
            addedHashtagsRowsList.append(deepCopy)
            i+=1
    return htList[0] # return the first hashtag

In [15]:
addedHashtagsRowsList = []
tw5_1 = tw5.copy()
pr('Multiplying the hashtag rows... (around 10 min)')
tw5_1['hashtag'] = tw5.apply(multiplyHashtagRows, args=[tw5.columns,], axis=1)
pr('Finished! {} rows will be added to the dataframe!'.format(len(addedHashtagsRowsList)))

10:06:00 Multiplying the hashtag rows... (around 10 min)
10:15:02 Finished! 3350600 rows will be added to the dataframe!


In [16]:
pr('Starting to make the new dataframe with additionnal rows..')
addedHashtagsDf = pd.DataFrame(addedHashtagsRowsList)
addedHashtagsDf.set_index(['createdAt'], inplace=True)
pr('Starting to append the two df... Old df size = {}'.format(len(tw5_1)))
tw6 = tw5_1.append(addedHashtagsDf)
pr('Done! New df size = {}'.format(len(tw6)))

10:15:02 Starting to make the new dataframe with additionnal rows..
10:15:16 Starting to append the two df... Old df size = 3875280
10:15:21 Done! New df size = 7225880


In [17]:
print('Example hahshtag:')
tw6[tw6['hashtag'] == addedHashtagsRowsList[0]['hashtag']].head(3)

Example hahshtag:


,followersCount,friendsCount,hashtag,id,inReplyTo,latitude,longitude,placeId,placeLatitude,placeLongitude,screenName,source,sourceName,sourceUrl,statusesCount,text,truncated,userId,userLocation,userName
createdAt,,,,,,,,,,,,,,,,,,,,
2011-07-24 16:03:13,1121,763.0,vfb,95162125584039936,9.501397e+16,NaN,NaN,e401fb8eb4e7595a,47.5356,9.14004,eLd0raDo,14,Tweetbot for iOS,http://tapbots.com/tweetbot,6735.0,"@mikstweed Wie lange ist es her, dass der #vfb...",NaN,921241,"Kreuzlingen, TG, Switzerland",Markus Tressl
2012-07-29 16:15:54,1008,767.0,vfb,229611214802669570,NaN,47.6683,9.48077,7e00050f2f9230bc,47.6684,9.47092,jens_nagler,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,1697.0,Man sollte ja nicht zu viel reininterpretieren...,NaN,5.71385e+07,Esslingen,Jens Nagler
2012-07-29 16:21:45,1008,767.0,vfb,229612686525231104,2.296124e+17,47.6682,9.48068,7e00050f2f9230bc,47.6684,9.47092,jens_nagler,1,Twitter for iPhone,http://twitter.com/#!/download/iphone,1697.0,"@LLcurly Das Besondere daran ist, dass nur zwe...",NaN,5.71385e+07,Esslingen,Jens Nagler


In [18]:
tw6.hashtag.head()

createdAt
2010-02-23 09:59:41               fb
2010-02-23 11:28:27              yam
2010-02-23 17:47:11               24
2010-02-23 18:19:03    iphoneography
2010-02-23 18:31:46     partnermonth
Name: hashtag, dtype: object

## Grouping per hashtags per day

In [19]:
tw6.dropna(subset=['longitude'], inplace=True)
tw6.dropna(subset=['latitude'], inplace=True)

In [20]:
tw6.latitude = tw6.latitude.apply(float)
tw6.longitude = tw6.longitude.apply(float)

In [21]:
delimiter = '_$$$_'
str_join = lambda x: delimiter.join(x)

Function that applies to a dataframe will group each row by day and aggregate all its content:

In [22]:
def aggDate(df):
    temp = df.groupby(df.index.map(lambda x: x.date))
    groupedDf = temp.agg({  'text' : str_join,
                            'longitude' : np.median,
                            'latitude' : np.median,
                            'hashtag' : lambda x: x.iloc[0], ## the first occurence
                            'numberOfTweets' : 'count',
                            'userId' : pd.Series.nunique})
    return groupedDf

In [23]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            if df['userId'][0] > 1:
                threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
                df['event'] = df.numberOfTweets.apply(lambda x: x > threshold and x > MIN_NB_TWEETS_DURING_EVENT)
            
            ## Remove recurrent events:
            detectedEventDf = df[df['event']]
            if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
                df['event'] = False
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

10:15:37 Making column number of tweets
10:15:37 Starting group by hastag...
10:15:37 Starting to put hashtag in dictionary... (around 50 min)
10:20:36 10%
10:25:15 20%
10:29:44 30%
10:34:19 40%
10:38:49 50%
10:43:21 60%
10:47:50 70%
10:52:26 80%
10:56:56 90%
11:01:23 100%
11:01:23 Finished operations! Dictionary with 607601 different hashtags.


In [24]:
# aggDate(gp.get_group('twibisg')).head(3)
# dictionary['twibisg'].head(3)
print('Dictionary with hashtags dataframes:')
dictionary[list(dictionary.keys())[5]].head(4)

Dictionary with hashtags dataframes:


,text,latitude,hashtag,longitude,numberOfTweets
2014-03-12,"Para la gente de #Trenton, NJ -: #Adidas #Sorp...",47.1379,sorprendente,9.52186,1
2014-04-07,Vamos #Celebrar Adidas #Achievement #Pendiente...,46.2058,sorprendente,6.14188,1
2014-04-11,Per le persone da #PapuaNewGuinea: #Adidas #Ta...,47.3843,sorprendente,8.52959,1


## Event detection (with elimination of recurrent events)

Parameters that define events:

In [25]:
## Parameters of an event:
MIN_TOT_NB_TWEETS = 20 ## The hashtag must have happened at least this number of times to be considered.
MIN_NB_DAYS_WITH_HASHTAGS = 3 ## The hashtags must appear at least this number of different days to be considered.
MIN_NB_TWEETS_DURING_EVENT = 7 ## To be considered an event, the hashtag must happen at least this nb of times during the day.
THRESHOLD_ANOMALY_FACTOR = 2.5 ## The occurence of a hashtag during a single day must be above the mean by this FACTOR
                             ## multiplied by the std to be considered as an anomaly.
MAX_DURATION_OF_EVENT = timedelta(days=30) ## The maximum number of days we consider an event can happen
MIN_DURATION_BEFORE_NEW_EVENT = timedelta(days=304) ## (= 10 months) The min time that should pass before an event can happen
                                                    ## again and still be considered as event (ie. Christmas is an event
                                                    ## each year)

Helper functions to detect recurrent events that should be removed:

In [26]:
def isSpecificEventListIllegal(detectedEventDateList):
    '''
    Return true if the list of dates contain illegal tupples of events, so if the event is recurrent
    which would mean it is not a real event.
    '''
    def datesAreIllegal(date1, date2, date3):
        '''
        Return true if the 3 dates are not to be considered as regular events.
        '''
        ## Return if the difference is too small to be considered as 2 different events
        def diffIsSmall(timeDiff):  
            return timeDiff < MAX_DURATION_OF_EVENT

        ## Return true if the difference is not big enough to be an annual event.
        def isDiffSuspect(timeDiff):
            return timeDiff < MIN_DURATION_BEFORE_NEW_EVENT   

        diff1 = abs(date1 - date2)
        diff2 = abs(date2 - date3)
        diff3 = abs(date3 - date1)

        ## The difference is too small, it must be the same event
        if diffIsSmall(diff1) or diffIsSmall(diff2) or diffIsSmall(diff3):
            return False

        ## If there are at least 2 out of 3 suspect difference, then the dates are illegal
        if isDiffSuspect(diff1):
            return isDiffSuspect(diff2) or isDiffSuspect(diff3)
        else:
            return isDiffSuspect(diff2) and isDiffSuspect(diff3)
    
    ## MAIN FUNCTION : ##
    # Go through the list of events and try all "triples" to see if there is any illegal triples. This is a quickly done
    # code to do that. Code complexity bellow is in O(k^3), with k being the size of the list. We will apply this function
    # to n list so we will have an overall complexity in O(n*k^3). We can consider however that each list will
    # be small so k can be considered as constant and therefore the overall complexity will be in O(n).
    for i in range(len(detectedEventDateList) - 2):
        for j in range(i, len(detectedEventDateList) - 1):
            for k in range(j, len(detectedEventDateList)):
                if datesAreIllegal(detectedEventDateList[i], detectedEventDateList[j], detectedEventDateList[k]):
                    return True
    return False

Main method to detect event according to all our criterias.

In [27]:
pr('Starting to compute {} dict items to detect event. (4 min)'.format(len(dictionary)))
nbOfEventDetected = 0
count = 0
printingValue = int(len(dictionary) / 10)
for [h,df] in dictionary.items():
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
    df['event'] = False
    if len(df) > MIN_NB_DAYS_WITH_HASHTAGS:
        if df['numberOfTweets'].sum() > MIN_TOT_NB_TWEETS:
            threshold = df['numberOfTweets'].mean() + THRESHOLD_ANOMALY_FACTOR * df['numberOfTweets'].std()
            df['event'] = df.numberOfTweets.apply(lambda x: x >= threshold and x >= MIN_NB_TWEETS_DURING_EVENT)
            
            ## Remove recurrent events:
            detectedEventDf = df[df['event']]
            if len(detectedEventDf) > 2 and isSpecificEventListIllegal(detectedEventDf.index):
                df['event'] = False
            nbOfEventDetected += len(df[df['event']])
pr('Finished! Number of events detected = {}'.format(nbOfEventDetected))

11:01:24 Starting to compute 607601 dict items to detect event. (4 min)
11:01:46 10%
11:02:08 20%
11:02:30 30%
11:02:52 40%
11:03:14 50%
11:03:36 60%
11:04:13 70%
11:04:35 80%
11:04:58 90%
11:05:20 100%
11:05:20 Finished! Number of events detected = 7028


## Merging close events and grouping into single event dataframe

Here, we have a function that is going to merge events that are considered as too "close" to each other to be considered individually.

In [28]:
def mergeCloseEvents(rowsList):
    '''
    Take a list of dictionary, where each dictionary is a "row" of the event df, which contained detected events.
    It will process the list to detect event that are close and merge them together.
    Return : the processed list of event.
    '''
    
    def areCloseEvents(event1, event2):
        '''
        Return true is 2 events dates are defined as "close"
        '''
        return abs(event1['date'] - event2['date']) < MAX_DURATION_OF_EVENT
        
    def mergeCloseEventsSublist(closeEventList):
        '''
        This will be applied to each close event sublist. It will merge all events into one unique event.
        The event will consist of the total number of tweets, with the concatenation of the tweet texts and the mean
        of longitude/latitude. A meanDate will be defined as a ponderated mean between all dates.
        The final date will be the one that is in the closeEventList and is closest to this mean date.
        We did this to keep the meaning of the date if it had some, and not have some meaningless "mean-date".
        '''
        latitude = 0
        longitude = 0
        numberOfTweets = 0
        text = ""
        originalDate = closeEventList[0]['date']
        dateDiff = timedelta(days=0)
        first = True        
        for tweet in closeEventList:
            longitude += tweet['longitude']
            latitude += tweet['latitude'] 
            numberOfTweets += tweet['numberOfTweets']
            if first:
                text = tweet['text']
                first = False
            else:
                text += delimiter + tweet['text']
                dateDiff = dateDiff + (tweet['date'] - originalDate) * tweet['numberOfTweets']

        ## It is multiplied by 2 then soustracted to round correctly to the nearest day
        meanDate = originalDate + 2* dateDiff / numberOfTweets - dateDiff / numberOfTweets        
        latitude = latitude / len(closeEventList)
        longitude = longitude / len(closeEventList)
        
        ## We are going to detect the event the closest to the mean date
        minSelectedDate = closeEventList[0]['date']
        minDistance = abs(closeEventList[0]['date'] - meanDate)
        for tweet in closeEventList:
            if abs(tweet['date'] - meanDate) < minDistance:
                minSelectedDate = tweet['date']    
        
        return {'date': minSelectedDate, 'hashtag': closeEventList[0]['hashtag'], 'text': text,
                    'longitude': longitude, 'latitude':latitude, 'numberOfTweets': numberOfTweets, }
    
    ############ -----  MAIN METHOD  ----- ############
    
    ## If the list is big enough, go through the list and form an export list and merge elements that needs to.
    if len(rowsList) < 2:
        return rowsList
    else:
        firstLastPosOfItemsToMerge = []
        sortedRowsList = sorted(rowsList, key=itemgetter('date')) 
        exportedEventList = []
        ## This goes through the *sorted* list and add the pair of indices (first indice and last indice) where events 
        ## that should be merged appear.
        lastEventWasClose = False
        firstItem = -1
        for i in range(0, len(sortedRowsList)-1):
            if areCloseEvents(sortedRowsList[i], sortedRowsList[i+1]):
                if not lastEventWasClose: # So it is the first pairs of the sublist of close events in the whole list
                    firstItem = i
                    lastEventWasClose = True
            else:
                if lastEventWasClose: # So the list has just ended.
                    exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:i+1]))
                    lastEventWasClose = False
                else: # The element is by itself, let's append it
                    exportedEventList.append(sortedRowsList[i])  
        if lastEventWasClose: # If there were events to merge till the last elem of list
            exportedEventList.append(mergeCloseEventsSublist(sortedRowsList[firstItem:len(sortedRowsList)]))
        else:
            exportedEventList.append(sortedRowsList[len(sortedRowsList)-1])
    
    return exportedEventList

This function will be applied to each dataframe. If a row is detected as an event, it will be added to the locaRowsList which will be used to make a general dataframe of all the events.

In [29]:
localRowsList = []
def applyToMakeEventDf(row):
    if row.event:
        rowToAdd = {'date': row.name, 'hashtag': row.hashtag, 'text': row.text,
                    'longitude': row.longitude, 'latitude':row.latitude, 'numberOfTweets': row.numberOfTweets, }
        global localRowsList
        localRowsList.append(rowToAdd)

In [30]:
eventRowsList = []
localRowsList = []
count = 0
printingValue = int(len(dictionary) / 10)

pr('Starting to make event df with {} dataframes. (around 6 min)'.format(len(dictionary)))
for h, df in dictionary.items():
    global localRowsList
    localRowsList = []
    count += 1
    if count % printingValue == 0:
        pr("{:.0f}%".format(count/len(dictionary)*100))
        
    df.apply(applyToMakeEventDf, axis=1)
    mergedList = mergeCloseEvents(localRowsList) # merging close events
    eventRowsList += mergedList

pr('Making new dataframe.')
new_events = pd.DataFrame(eventRowsList)
new_events.set_index(['date'], inplace=True)
pr('Finished! Dataframe with {} rows'.format(len(new_events)))

11:05:20 Starting to make event df with 607601 dataframes. (around 6 min)
11:05:52 10%
11:06:24 20%
11:06:56 30%
11:07:29 40%
11:08:01 50%
11:08:33 60%
11:09:05 70%
11:09:37 80%
11:10:09 90%
11:10:41 100%
11:10:41 Making new dataframe.
11:10:41 Finished! Dataframe with 5436 rows


In [41]:
print('Events dataframe:')
new_events.head(50)

Events dataframe:


,hashtag,latitude,longitude,numberOfTweets,text
date,,,,,
2015-10-26,cigars,47.36630,8.451350,10,#Swiss #HabanosDay 2015 #hostess #Cuban #cigar...
2012-09-30,atheism,46.46470,6.898390,8,"I'm very happy to be a gentile, an infidel, an..."
2014-03-09,carnival,46.37285,7.990435,8,#carnival #party (@ Ex Jolly Roger) http://t.c...
2015-03-03,instacars,46.23390,6.118480,14,#Opel #Karl #instacar #instacars #cars #cartas...
2011-05-27,velo,47.14380,8.896060,13,20 % progress …Pause! #Velo #Heimfahrt_$$$_25 ...
2016-06-14,αυτοκόλλητηετικέτα,47.56510,7.592140,23,#sticker #oblakin #nálepka #αυτοκόλλητηετικ...
2015-08-05,08,46.57900,8.421210,13,#05#08#2015 #Selkingen #Valais#Wallis #TravelD...
2016-08-13,08,46.66020,8.670280,19,#13#08#2016 #ontheroadagain #TripAroundSwitzer...
2014-06-15,bih,47.05340,7.448055,58,#BIH #BIH #BIH #BIH #BIH #BIH #BIH #BIH #BIH #...


In [32]:
print('Linked dataframe of all days:')
dictionary[new_events.iloc[0].hashtag].head(10)

Linked dataframe of all days:


,text,latitude,hashtag,longitude,numberOfTweets,event
2012-08-29,This #RyJ #CedrosDeLuxeNo1 from the 70s is one...,47.4062,cigars,9.60897,1,False
2012-10-24,@HabanoSy @PerfectSmokes For #cigars which're ...,47.4063,cigars,9.60882,1,False
2012-12-26,on avait dit rendez-vous chez Zyno... #davidof...,46.2727,cigars,6.13460,1,False
2012-12-27,#cigare #cigares #cigare #cigars #davidoff @ G...,46.2727,cigars,6.13460,1,False
2013-01-26,#Coffee to start the day. #noStarbucks #Cigars...,47.4062,cigars,9.60882,1,False
2013-02-08,#Cigars #collage #Diplomaticos #Montecristo #C...,47.4062,cigars,9.60882,1,False
2013-02-16,#Partagas #collage #cigars #Cigarians @ Casa B...,47.4062,cigars,9.60882,1,False
2013-03-05,#Cigars #LaGloriaCubana #MedailledOr #No3 #Cig...,47.4062,cigars,9.60882,1,False
2013-03-20,Does that mean #cigars are allowed!!!! #cigari...,46.4446,cigars,6.21659,1,False
2013-03-30,Tonite's choice of #cigars: #Cohiba #35Anivers...,47.4062,cigars,9.60882,1,False


## Exporting data

In [33]:
total_number_of_events = len(new_events)
print('There are {} events.'.format(total_number_of_events))

There are 5436 events.


In [34]:
e_df = new_events.copy()
e_df['date'] = e_df.index
e_df.index = [i for i in range (len(e_df))]
e_df.head(1)

,hashtag,latitude,longitude,numberOfTweets,text,date
0,cigars,47.3663,8.45135,10,#Swiss #HabanosDay 2015 #hostess #Cuban #cigar...,2015-10-26


We are going to generate the right datetimes for the jsons:

In [35]:
epoch_dt = datetime(1970, 1, 1)
def to_utc(date):
    d_dt = datetime.combine(date, datetime.min.time())
    return int((d_dt - epoch_dt).total_seconds()*1000)

In [36]:
def convert_to_unix_time(record):
    datetime_index = pd.DatetimeIndex([datetime(record['year'], record['month'], 1)])
    unix_time_index = datetime_index.astype(np.int64) // 10**6
    return unix_time_index[0]

In [37]:
pr('Converting dates...')
e_df['year'] = e_df['date'].apply(lambda x: x.year)
e_df['month'] = e_df['date'].apply(lambda x: x.month)
e_df['utc_date'] = e_df['date'].apply(lambda x: to_utc(x))
e_df['unix_time'] = e_df.apply(convert_to_unix_time, axis=1)
pr('Done.')
e_df.head(1)

11:10:41 Converting dates...
11:10:42 Done.


,hashtag,latitude,longitude,numberOfTweets,text,date,year,month,utc_date,unix_time
0,cigars,47.3663,8.45135,10,#Swiss #HabanosDay 2015 #hostess #Cuban #cigar...,2015-10-26,2015,10,1445817600000,1443657600000


Grouping by months and generating the dictionary for the json.

In [38]:
e_gb_month = e_df.groupby(e_df.unix_time)

In [39]:
pr('Making event list...')
months = []
for month, df in e_gb_month:
    days = []
    for i in range (len(df)):
        ht = df.iloc[i]['hashtag']
        lat = df.iloc[i]['latitude']
        lon = df.iloc[i]['longitude']
        t_num = df.iloc[i]['numberOfTweets']
        tweets = df.iloc[i]['text'].split(delimiter)
        date = df.iloc[i]['utc_date']
        
        data_unit = { 'name': ht
                    , 'latitude' : lat
                    , 'longitude' : lon
                    , 'tweets' : tweets
                    , 'number_of_tweets' : str(t_num)
                    , 'date' : int(date)}
        days.append(data_unit)
    
    curr_month = {'date': int(month), 'data' : days}
    months.append(curr_month)

final_events = {'events' : months}
pr('Done.')

11:10:42 Making event list...
11:10:48 Done.


In [40]:
exportFilename = 'export_twitter_events_' + datetime.now().strftime("%Y-%m-%d_%Hh%Mmin%S") + \
'_' + str(total_number_of_events)+ '_events.json'
exportPath =  os.path.join('data', exportFilename)

pr('Exporting to json...')
with open(exportPath, 'w') as f:
     json.dump(final_events, f)
pr('Export done. File "{}" has been created.'.format(exportFilename))

11:10:48 Exporting to json...
11:10:48 Export done. File "export_twitter_events_2017-01-31_11h10min48_5436_events.json" has been created.
